In [61]:
from pynq import Overlay
import numpy as np
import random
from pynq import allocate
import matplotlib.pyplot as plt
from scipy.io import wavfile
from time import sleep

# Functions

In [14]:
FILE_1 = "E7_chord_trimmed.wav"
FILE_2 = "E_string_sample.wav"

def read_wav(filename):
    return wavfile.read(filename)

def plot_wav(sample_rate, data):
    length = data.shape[0] / sample_rate
    
    # prints the information from the file
    print(f"Recording length: {length:.3f}s")
    print(f"Sample Rate: {sample_rate}Hz")
    
    # creates a time array for plotting
    time = np.linspace(0., length, data.shape[0])
    
    # uses matplot.lib for plotting functions, plotting both audio channels
    plt.plot(time, data[:, 0], label="Left channel")
    plt.plot(time, data[:, 1], label="Right channel")
    plt.legend()
    plt.xlabel("Time [s]")
    plt.ylabel("Amplitude")
    plt.show()

def do_fft(signal, sample_rate):
    fft_spectrum = np.fft.rfft(signal)
    freq = np.fft.rfftfreq(signal.size, d=1./sample_rate)  # get frequency values of FFT
    fft_spectrum_abs = np.abs(fft_spectrum)  # compute absolute value of FFT plot
    return freq, fft_spectrum_abs

class Max:
    def __init__(self):
        self.ampl = 0
        self.freq = 0
        self.lwr_bnd = 0
        self.upr_bnd = 0
    
    def calc_bounds(self, freq):
        self.lwr_bnd = freq - ((freq * 0.0280645232089) - 0.000609838013018)
        self.upr_bnd = freq + ((freq * 0.0297331943366) - 0.000226403391093)
    
    def in_freq_rng(self, freq) -> bool: 
        return self.lwr_bnd < freq < self.upr_bnd
            
def get_top_max(freq_array, fft_array, n):
    max_arr = [Max() for _ in range(n)]
    for i in range(len(freq_array)):
        compare_ampl = fft_array[i]
        compare_freq = freq_array[i]
        
        swap = False
        for _ in range(n):
            if max_arr[_].in_freq_rng(compare_freq):
                swap = True
            if max_arr[_].ampl < compare_ampl:
                temp_ampl = max_arr[_].ampl
                temp_freq = max_arr[_].freq
                max_arr[_].ampl = compare_ampl
                max_arr[_].freq = compare_freq
                max_arr[_].calc_bounds(compare_freq)
                compare_ampl = temp_ampl
                compare_freq = temp_freq
                if swap or any([max_arr[z].in_freq_rng(temp_freq) for z in range(n)]):
                    break
            if swap:
                break
    return [max_arr[_].freq for _ in range(n)]

def get_lwr_bnd(freq):
    return freq - ((freq * 0.0280645232089) - 0.000609838013018)


def get_upr_bnd(freq):
    return freq + ((freq * 0.0297331943366) - 0.000226403391093)


def overlap(base_freq, compare_freq) -> bool:
    for i in range(9):
        lwr_bnd = get_lwr_bnd(base_freq) * (2 ** i)
        upr_bnd = get_upr_bnd(base_freq) * (2 ** i)
        in_rng = (lwr_bnd < compare_freq) and (compare_freq < upr_bnd)
        if in_rng:
            return True
    return False


def get_top_3(fft_array):
    max_ampl_1 = 0
    max_freq_1 = 0
    max_ampl_2 = 0
    max_freq_2 = 0
    max_ampl_3 = 0
    max_freq_3 = 0

    temp_ampl = 0
    temp_freq = 0
    swap = False

    for i in range(len(fft_array)):

        swap = False
        comp_ampl = fft_array[i]
        comp_freq = i

        if max_ampl_1 < comp_ampl:
            if overlap(max_freq_1, comp_freq):
                swap = True
            temp_ampl = max_ampl_1
            temp_freq = max_freq_1
            max_ampl_1 = comp_ampl
            max_freq_1 = comp_freq
            comp_ampl = temp_ampl
            comp_freq = temp_freq
            if swap:
                continue
        elif overlap(max_freq_1, comp_freq):
            continue

        if max_ampl_2 < comp_ampl:
            if overlap(max_freq_2, comp_freq):
                swap = True
            temp_ampl = max_ampl_2
            temp_freq = max_freq_2
            max_ampl_2 = comp_ampl
            max_freq_2 = comp_freq
            comp_ampl = temp_ampl
            comp_freq = temp_freq
            if swap:
                continue
        elif overlap(max_freq_2, comp_freq):
            continue

        if max_ampl_3 < comp_ampl:
            max_ampl_3 = comp_ampl
            max_freq_3 = comp_freq

    return max_freq_1, max_freq_2, max_freq_3

def get_note(pitch_freq, exact=True, split=False):
    CONCERT_PITCH = 440  # tuning standard in music
    ALL_NOTES = ["A", "A#", "B", "C", "C#", "D", "D#", "E", "F", "F#", "G", "G#"]  # musical notes arranged properly
    i = int(np.round(np.log2(pitch_freq / CONCERT_PITCH) * 12))
    ret = (ALL_NOTES[i % 12], 4 + (i + 9) // 12) if split else ALL_NOTES[i % 12] + (str(4 + (i + 9) // 12) if exact else "")
    return ret

# def get_chord(freq):
#     notes = [get_note(i, exact=False) for i in freq]
#     notes = [x for _, x in sorted(zip(freq, notes))]
#     for n in notes[:]:
#         if 'a' in n.lower() or 'b' in n.lower():
#             notes.remove(n)
#             notes.append(n)
#     triads = chords.determine(notes)
#     if triads:
#         return triads[0]
#     return None

def read_fft_file():
    with open("fftdata.txt", 'r') as f:
        data = [float(_) for _ in f.readlines()]
    return data

# Setup

In [89]:
sample_rate, data = read_wav(FILE_1)
freq, fft = do_fft(data[22050:66150,0], sample_rate)

# plot_wav(sample_rate, data)

SAMPLES = 1024

# MAX

## Software

In [86]:
def max_sw():
    return get_top_max(freq[:SAMPLES], fft[:SAMPLES], 1)

In [87]:
freq = max_sw()
print(f"Recorded Note: {get_note(freq[0])} ({freq[0]}Hz)")

Recorded Note: B2 (124.0Hz)


In [88]:
%timeit max_sw()

10000 loops, best of 3: 132 µs per loop


## Hardware

In [93]:
max_overlay = Overlay("/home/xilinx/pynq/overlays/maxVal/maxValue2.bit")
max_ip = max_overlay.max_DMA.get_top_freq_0
max_dma = max_overlay.max_DMA.axi_dma_0
input_buffer = allocate(shape=SAMPLES, dtype=np.csingle)
input_buffer[:] = fft[:SAMPLES]

In [94]:
def max_hw():
    max_dma.sendchannel.transfer(input_buffer)
    max_dma.sendchannel.wait()
    return max_ip.read(0x10)

In [95]:
freq = max_hw()
print(f"Recorded Note: {get_note(freq)} ({freq:}Hz)")

Recorded Note: B2 (124Hz)


In [96]:
%timeit max_hw()

1000 loops, best of 3: 312 µs per loop


# TOP

## Software

In [56]:
def top_sw():
    return get_top_3(fft[:SAMPLES])

In [57]:
freq = top_sw()
for f in freq:
    print(f"Recorded Note: {get_note(f)} ({f:}Hz)")

Recorded Note: B2 (124Hz)
Recorded Note: E2 (82Hz)
Recorded Note: G#3 (209Hz)


In [58]:
%timeit top_sw()

1 loop, best of 3: 205 ms per loop


## Hardware

In [64]:
top_overlay = Overlay("/home/xilinx/pynq/overlays/top_overlay/top_overlay/top_3.bit")
top_ip = top_overlay.get_top_3_0
top_dma = top_overlay.axi_dma_0

input_buffer = allocate(shape=(SAMPLES,), dtype=np.csingle)
input_buffer[:] = fft[:SAMPLES]

In [65]:
def top_hw(scale_factor=1):
    top_ip.write(0x10, int(scale_factor))
    top_dma.sendchannel.transfer(input_buffer)
    top_dma.sendchannel.wait()
    return top_ip.read(0x18), top_ip.read(0x20), top_ip.read(0x28)

In [66]:
freq = top_hw()
for f in freq:
    print(f"Recorded Note: {get_note(f)} ({f:}Hz)")

Recorded Note: B2 (124Hz)
Recorded Note: E2 (82Hz)
Recorded Note: G#3 (209Hz)


In [67]:
%timeit top_hw()

100 loops, best of 3: 8.52 ms per loop
